---
# Lagrange Multipliers
---

In [ ]:
using LinearAlgebra
using Plots, LaTeXStrings
using Printf

$$
\left\{
\begin{array}{ll}
\text{minimize} & \frac{1}{2} \|Ax - b\|_2^2 \\
\text{subject to} & \frac{1}{2}\|x\|_2^2 = \frac{1}{2}
\end{array}
\right.
$$

$$
f(x) = \frac{1}{2} \|Ax - b\|_2^2, \qquad g(x) = \frac{1}{2} - \frac{1}{2}\|x\|_2^2
$$

In [ ]:
m, n = 8, 2

A = rand(-10:10, m, n)
x̄ = [2.0, 2.0]

b = A*x̄

In [ ]:
norm(b - A*x̄)

In [ ]:
f(x) = 0.5*norm(b - A*x)^2
f(x,y) = f([x,y])

In [ ]:
∇f(x) = A'*(b - A*x)

In [ ]:
f(x̄)

In [ ]:
norm(x̄)

In [ ]:
g(x) = 0.5 - 0.5*norm(x)^2
g(x,y) = g([x,y])

In [ ]:
∇g(x) = -x

In [ ]:
g(x̄)

In [ ]:
xx = -2:0.01:4
yy = -2:0.01:4
flevels = [20, 50, 100, 200, 500, 1000, 2000]

plot(xlabel=L"x", ylabel=L"y", aspect_ratio=:equal, colorbar=:none, size=(600,600))
contour!(xx, yy, f, levels=flevels, color=:black, contour_labels=true)
contour!(xx, yy, g, levels=[0], color=:red, contour_labels=false)
scatter!([x̄[1]], [x̄[2]], c=:black, label=:none)

---

# JuMP.jl

In [ ]:
using JuMP, Ipopt

In [ ]:
model = Model(Ipopt.Optimizer)
set_optimizer_attribute(model, MOI.Silent(), true)

@variable(model, x[1:2])
@objective(model, Min, dot(A*x - b, A*x - b)/2)
@constraint(model, con, dot(x,x)/2 == 1//2)

model

In [ ]:
results = optimize!(model)

xopt = value.(x)

In [ ]:
norm(xopt)

In [ ]:
optval = objective_value(model)

In [ ]:
λ = getdual(con)

In [ ]:
p = ∇f(xopt)

In [ ]:
q = ∇g(xopt)

In [ ]:
dot(p,q)/norm(p)/norm(q)

$$
\nabla f(x^*) = \lambda \nabla g(x^*)
$$

In [ ]:
∇f(xopt) - λ*∇g(xopt)

In [ ]:
xx = -2:0.01:4
yy = -2:0.01:4
flevels = [20, 50, 100, 200, 600, 1000, 2000, optval]

p = p/norm(p)
q = q/norm(q)

plot(xlabel=L"x", ylabel=L"y", aspect_ratio=:equal, colorbar=:none, size=(600,600))
contour!(xx, yy, f, levels=flevels, color=:black, contour_labels=true)
contour!(xx, yy, g, levels=[0], color=:red, contour_labels=false)
quiver!([xopt[1]], [xopt[2]], quiver=([p[1]],[p[2]]), label=:none, c=:black)
quiver!([xopt[1]], [xopt[2]], quiver=([q[1]],[q[2]]), label=:none, c=:red)
scatter!([x̄[1]], [x̄[2]], c=:black, label=:none)
scatter!([xopt[1]], [xopt[2]], c=:red, label=:none)

---

# Lagrange multiplier = derivative of the optimal value

In [ ]:
function optvalfun(ρ)
    model = Model(Ipopt.Optimizer)
    set_optimizer_attribute(model, MOI.Silent(), true)

    @variable(model, x[1:2])
    @objective(model, Min, dot(A*x - b, A*x - b)/2)
    @constraint(model, con, dot(x,x)/2 == ρ)
    
    optimize!(model)
    
    return objective_value(model)
end

In [ ]:
plot(xlabel=L"\rho", ylabel=L"y")
plot!(optvalfun, 0, 13, label=L"y = v(\rho)")
plot!(ρ -> optval + λ*(ρ - 1/2), 0, -optval/λ + 1/2, label=:none)
scatter!([1/2], [optval], c=1, label=:none)

In [ ]:
0.5*norm(x̄)^2

In [ ]:
optvalfun(4.0)